In [1]:
import pandas as pd
import numpy as np
from keras.datasets import boston_housing 
from keras import models, layers, optimizers


In [6]:
#DESCARGANDO DATOS
(train_data , train_targets) ,(test_data,test_targets) = boston_housing.load_data()

train_data[1]


array([2.1770e-02, 8.2500e+01, 2.0300e+00, 0.0000e+00, 4.1500e-01,
       7.6100e+00, 1.5700e+01, 6.2700e+00, 2.0000e+00, 3.4800e+02,
       1.4700e+01, 3.9538e+02, 3.1100e+00])

In [7]:
train_targets[1]

42.3

In [8]:
#NORMALIZACION

#media
mean = train_data.mean(axis=0)
#train data - media
train_data = train_data  - mean
#desviacion estandar
std = train_data.std(axis=0)
#train data / desviacion estandar
train_data = train_data / std

#lo mismo , fijarte que usas la media del train data !!jamas uses data de test en este caso
test_data = test_data - mean
test_data = test_data / std


In [9]:
#DEFINIENDO RED NEURONAL
def build_model_regression(input_data):
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(input_data,)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
    return model

In [12]:
#K-FOLD VALIDATION
#

k = 4
num_val_samples = len(train_data) // 4
num_epoch = 80
all_history = []



In [13]:
for i in range(k):
    print("Fold " , i)
    val_data = train_data[i*num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
     train_data[(i+1) * num_val_samples:]],
     axis= 0   
    )
    
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
     train_targets[(i+1) * num_val_samples:]],
     axis= 0   
    )    
    model = build_model_regression(13)
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epoch, batch_size =16, 
                        validation_data = (val_data, val_targets),
                        verbose=0)
    all_history.append(history.history['val_mae'])
    

Fold  0
Fold  1
Fold  2
Fold  3


In [15]:
#Media de todos los mae
len(all_history[0])

all_mae_avg = pd.DataFrame(all_history).mean(axis=0)